# GCC118 - Programação Matemática
## Prof. Mayron César de O. Moreira
### Programação Linear: Modelagem Matemática

- *Universidade Federal de Lavras*  
- *Departamento de Ciência da Computação*  

# Problema da empresa *Minas Máquinas*

Considere uma empresa que fabrica $n$ produtos e deseja programar sua produção nos próximos $T$ períodos. Cada produto possui uma demanda específica em cada período, que deve ser exatamente atendida. Os custos de produção (estoque) dependem da quantidade produzida (estocada) e variam de acordo com o item e período. A produção de um determinado item consome uma certa quantidade de recurso (por exemplo: água, energia, gás, etc) pré-determinado para o período em questão. Diante deste cenário, a *Minas Máquinas* deseja planejar o controle da produção dos produtos em cada período, visando a minimização dos custos de produção e custos de estocagem.

## Modelagem matemática

A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão. 

### Parâmetros

- $n$: quantidade de produtos;
- $T$: quantidade de períodos;
- $d_{it}$: demanda do item $i$ no período $t$, $i=1,...,n$ e $t=1,...,T$;
- $R_t$: disponibilidade de recursos (renováveis) no período $t$;
- $r_i$: quantidade de recursos necessários para a produção de uma unidade do item $i$;
- $c_{it}$: custo para produzir uma unidade do item $i$ no período $t$;
- $h_{it}$: custo de estocar uma unidade do item $i$ no período $t$.

### Variáveis

- $x_{it} \ge 0$: quantidade do item $i$ produzido no período $t$;
- $I_{it} \ge 0$: quantidade do item $i$ estocado no fim do período $t$.

### Função objetivo

Minimização dos custos de produção e estoque de todos os itens, em todos os períodos.

\begin{equation}
\min \sum_{i=1}^{n}\sum_{t=1}^T c_{it}x_{it} + \sum_{i=1}^{n}\sum_{t=1}^T h_{it}I_{it}
\end{equation}

### Restrições

- Restrição 1 (Conservação de estoque e atendimento de demanda): o nível do estoque no período $t$ é igual ao que se tinha no período anterior, adicionado do que foi produzido agora, menos o que foi demandado em $t$. Esta igualdade garante o atendimento exato da demanda do período $t$.

\begin{equation}
I_{i,t-1} + x_{it} - d_{it}  = I_{it} \qquad i=1,...,n,t=1,...,T
\end{equation}

- Restrição 2: a produção dos itens não deve superar a capacidade (disponibilidade) de recursos da fábrica, em cada período $t$.
\begin{equation}
\sum_{i=1}^{n}r_{i}x_{it} \le R_t \qquad t=1,...,T
\end{equation}


### Modelo

\begin{equation}
\min f(x,I) = \sum_{i=1}^{n}\sum_{t=1}^T c_{it}x_{it} + \sum_{i=1}^{n}\sum_{t=1}^T h_{it}I_{it}
\end{equation}

sujeito a:

\begin{alignat}{2}
I_{i,t-1} + x_{it} - I_{it} = d_{it} && \qquad &i=1,...,n,t=1,...,T\\
\sum_{i=1}^{n}r_{i}x_{it} \le R_t && \qquad &t=1,...,T\\
x_{it} \ge 0 && \qquad &i=1,...,n,t=1,...,T\\
I_{it} \ge 0 && \qquad &i=1,...,n,t=1,...,T\\
\end{alignat}

## Implementação em Python

Apresentamos, a seguir, uma sugestão de implementação deste modelo em Python.

### Importação da biblioteca do Gurobi

In [1]:
from gurobipy import *

### Criação do modelo

In [2]:
# Criacao do modelo
m = Model("Minas_Maquinas")

Academic license - for non-commercial use only


### Declaração dos parâmetros do problema

Os valores abaixo são arbitrários, apenas para ilustrar o funcionamento do modelo.

In [3]:
# Numero de itens
n = 3

# Numero de periodos
T = 2

# Matriz de custos de producao
c = [[100,80],
     [60,80],
     [30,200]]

# Matriz de custos de estoque
h = [[2,2.5],
     [3,3.5],
     [3.5,3.5]]

# Vetor de recursos de cada item
r = [0.25,0.3,0.3]

# Vetor de recursos dos periodos
R = [200,250]

# Demanda dos itens em cada periodo
d = [[50,20],
     [40,60],
     [100,80]]

### Criação das variáveis de decisão

In [4]:
# Declaracao das variaveis x: quantidade produzida do item i no periodo t 
x = m.addVars(n, T, lb=0, ub=GRB.INFINITY, name="x")

# Declaracao das variaveis I: quantidade estocada do item i no fim do periodo t
I = m.addVars(n, T, lb=0, ub=GRB.INFINITY, name="I")

### Criação da função objetivo

In [5]:
m.setObjective(sum(c[i][t]*x[i,t] for i in range(n) for t in range(T)) 
               + sum(h[i][t]*I[i,t] for i in range(n) for t in range(T)), GRB.MINIMIZE)

### Restrição 1: Conservação de estoque e atendimento de demanda

In [6]:
# Conservacao de estoque e atendimento as demandas
m.addConstrs((I[i,t-1] + x[i,t] - I[i,t] == d[i][t]
    for i in range(n) for t in range(1,T)),"Demanda")

# A mesma restricao considerando o periodo 0 (não existe período anterior ao 0)
m.addConstrs((x[i,0] - I[i,0] == d[i][0] for i in range(n)), "Demanda0")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

### Restrição 2: Respeito a capacidade de cada período

In [7]:
m.addConstrs((sum(r[i]*x[i,t] for i in range(n)) <= R[t] for t in range(0,T)), "Capacidade")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

### Salva o modelo no formato .lp (importante para conferir se a formulação está implementada corretamente)

In [8]:
m.write('Minas_Maq.lp')

### Resolve o modelo

In [9]:
m.optimize()

Optimize a model with 8 rows, 12 columns and 21 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 5 rows, 9 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0400000e+04   3.750000e+01   0.000000e+00      0s
       3    1.8460000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds
Optimal objective  1.846000000e+04


### Imprimindo o valor das variáveis do problema

In [10]:
m.printAttr('X')


    Variable            X 
-------------------------
      x[0,0]           50 
      x[0,1]           20 
      x[1,0]          100 
      x[2,0]          180 
      I[1,0]           60 
      I[2,0]           80 


### Imprime o valor da função objetivo

In [11]:
print("Obj: " + str(m.objVal))

Obj: 18460.0


### Impressao do status da solucao

In [13]:
status = 'Status: '
if m.status == GRB.OPTIMAL:
    status += 'OTIMA'

elif m.status == GRB.INFEASIBLE:
    status += 'INFACTIVEL'

elif m.status == GRB.UNBOUNDED:
    status += 'ILIMITADA'

else:
    status += 'FACTIVEL'

print(status)

Status: OTIMA


## Referências

- Arenales, M., Armentano, V.; Morabito, R.; Yanasse, H. (2007). Pesquisa Operacional, 1a ed., Rio de Janeiro: Campus - Elsevier.